In [ ]:
#| default_exp core

# Core API
> API source and details

In [ ]:
#| export
import platform,gc
from fastcore.utils import *
from contextlib import closing
from pathlib import Path

from anki.collection import Collection
from anki import _backend
from anki.sync_pb2 import SyncAuth
from anki.errors import SyncError
from anki.models import ModelManager
from anki.notetypes_pb2 import NotetypeNameId
from anki.decks import DeckManager
from anki.decks_pb2 import DeckTreeNode, DeckNameId
from anki.cards import Card
from anki.notes import Note
from anki.collection_pb2 import OpChangesWithCount
from anki.sync_pb2 import SyncCollectionResponse

In [ ]:
user = os.environ['ANKI_USER']
passw = os.environ['ANKI_PASS']

## OO API

In [ ]:
#| export
def data_path():
    "Return the default Anki data folder for this OS."
    syst = platform.system()
    path = Path(os.environ['APPDATA']) if syst=='Windows' else Path.home()
    return path/('.local/share/Anki2' if syst=='Linux'
        else 'Library/Application Support/Anki2' if syst=='Darwin'
        else 'Anki2')

In [ ]:
data_path()

Path('/Users/racheltho/Library/Application Support/Anki2')

In [ ]:
data_path().ls()

[Path('/Users/racheltho/Library/Application Support/Anki2/User 1')]

In [ ]:
#| export
def profiles():
    "List available Anki profile names."
    dp = data_path()
    if not dp.exists(): return []
    return [p.name for p in dp.iterdir() if (p/'collection.anki2').exists()]

In [ ]:
profiles()

['User 1']

In [ ]:
#| export
# quiet down latency complaints
_backend.main_thread = lambda: None

In [ ]:
#| export
anki_defaults = dict(profile='User 1', model='Basic', deck='Default')

In [ ]:
#| export
@patch(cls_method=True)
def open(cls:Collection, profile=None):
    "Open a collection by profile name (creates if needed)."
    profile = profile or anki_defaults['profile']
    if not hasattr(cls, '_backend'): cls._backend = _backend.RustBackend()
    try: cls._backend.close_collection(downgrade_to_schema11=False)
    except: pass
    path = data_path() / profile
    path.mkdir(parents=True, exist_ok=True)
    col = cls(str(path / 'collection.anki2'), backend=cls._backend)
    col.profile_path = path
    return col

Class method to open a collection by profile name. Handles backend reuse, creates profile folder if needed, and stores `profile_path` for auth persistence.

In [ ]:
col = Collection.open()
col.profile_path

Path('/Users/racheltho/Library/Application Support/Anki2/User 1')

In [ ]:
#| export
@patch(as_prop=True)
def _auth_path(self:Collection): return self.profile_path / 'sync_auth.bin'

@patch
def save_auth(self:Collection, auth):
    "Persist SyncAuth to profile folder."
    self._auth_path.write_bytes(auth.SerializeToString())

@patch
def load_auth(self:Collection):
    "Load SyncAuth from profile folder, or None if not found."
    p = self._auth_path
    if not p.exists(): return None
    auth = SyncAuth()
    auth.ParseFromString(p.read_bytes())
    return auth

Auth persistence methods: `save_auth` writes the `SyncAuth` protobuf to disk, `load_auth` reads it back. The endpoint is crucial — AnkiWeb may redirect you to a different sync server.

In [ ]:
#| export
@patch
def sync(self:Collection, user=None, passw=None, media=True, upload=False):
    auth = self.load_auth()
    if not auth or not auth.endpoint:
        if not (user and passw): raise ValueError("No saved auth; provide user and passw")
        auth = self.sync_login(username=user, password=passw, endpoint=None)
    status = self.sync_status(auth)
    if status.new_endpoint: auth = SyncAuth(hkey=auth.hkey, endpoint=status.new_endpoint)
    self.save_auth(auth)
    if status.required == status.Required.NO_CHANGES: return
    try: result = self.sync_collection(auth, sync_media=media)
    except SyncError:
        if not (user and passw): raise
        auth = self.sync_login(username=user, password=passw, endpoint=auth.endpoint)
        result = self.sync_collection(auth, sync_media=media)
    if result.new_endpoint: auth = SyncAuth(hkey=auth.hkey, endpoint=result.new_endpoint)
    self.save_auth(auth)
    needs_full = (result.ChangesRequired.FULL_SYNC, result.ChangesRequired.FULL_DOWNLOAD, result.ChangesRequired.FULL_UPLOAD)
    if result.required in needs_full:
        do_upload = upload if result.required == result.ChangesRequired.FULL_SYNC else result.required == result.ChangesRequired.FULL_UPLOAD
        self.close_for_full_sync()
        self.full_upload_or_download(auth=auth, server_usn=result.server_media_usn, upload=do_upload)
        self.reopen(after_full_sync=True)
    else: return status
    return result

In [ ]:
note = col.newNote()
note.fields[0] = "Test card front"
note.fields[1] = "Test card back"
col.addNote(note)

1

In [ ]:
#| export
@patch
def _repr_markdown_(self:SyncCollectionResponse):
    req = self.ChangesRequired.DESCRIPTOR.values_by_number[self.required].name
    return f"**Sync**: {req or 'Normal'}"

In [ ]:
result = col.sync(user=user, passw=passw)
result

required: NORMAL_SYNC

Full sync implementation: loads saved auth (or logs in), handles endpoint changes, re-authenticates on failure, and manages full upload/download when required.

```python
# First sync requires credentials
col.sync(user=user, passw=passw)

# Subsequent syncs use saved auth
col.sync()
```

In [ ]:
#| export
def close_all():
    "Close all open Anki backends (nuclear option for crash recovery)."
    for obj in gc.get_objects():
        if isinstance(obj, RustBackend):
            try: obj.close_collection(downgrade_to_schema11=False)
            except: pass

Nuclear option for crash recovery — scans all Python objects to find and close any open Rust backends.

In [ ]:
#| export
@patch
def _repr_markdown_(self:ModelManager): return '\n'.join('- '+m.name for m in self.all_names_and_ids())

@patch
def _repr_markdown_(self:NotetypeNameId): return f"**{self.name}** (id: {self.id})"

In [ ]:
col.models

- Basic
- Basic (and reversed card)
- Basic (optional reversed card)
- Basic (type in the answer)
- Cloze
- Image Occlusion

In [ ]:
#| export
@patch
def _repr_markdown_(self:DeckManager): return '\n'.join('- '+d.name for d in self.all_names_and_ids())

@patch
def _repr_markdown_(self:DeckNameId): return f"**{self.name}** (id: {self.id})"

@patch
def _repr_markdown_(self:DeckTreeNode):
    return f"**{self.name}**: {self.new_count} new, {self.learn_count} learn, {self.review_count} review"

Markdown representations for decks. `DeckManager` lists deck names, `DeckNameId` shows name/id, `DeckTreeNode` shows name with due counts.

In [ ]:
col.sched.deck_due_tree().children[0]

**Default**: 11 new, 0 learn, 5 review

In [ ]:
col.decks

- Default
- Precalculus

Enables dict-style access: `col.models['Basic']` returns the model dict, `col.decks['Default']` returns the deck id.

In [ ]:
#| export
ModelManager.__getitem__ = ModelManager.by_name

In [ ]:
mdl = col.models['Basic']
mdl['id']

1764652425113

In [ ]:
#| export
@patch
def _repr_markdown_(self:Note):
    fields = ' | '.join(f"**{k}**: {self[k]}" for k in self._fmap)
    return f"{fields} | 🏷️ {', '.join(self.tags)}" if self.tags else fields

@patch
def __repr__(self:Note):
    fields = ', '.join(f"{k}={self[k]!r}" for k in self._fmap)
    return f"Note({self.id}, {fields}, tags={self.tags})"

In [ ]:
note = col.new_note(mdl)
note['Front'], note['Back'] = 'hola', 'hello'
note

**Front**: hola | **Back**: hello

In [ ]:
#| export
@patch
def _repr_markdown_(self:OpChangesWithCount): return f"✓ {self.count} change(s)"

In [ ]:
#| export
class Deck:
    def __init__(self, col, name):
        self.col, self.name = col, name
        self.id = col.decks.id_for_name(name)
    
    def add(self, model=None, tags=None, **fields): return self.col.add(model=model, deck=self.name, tags=tags, **fields)
    
    @property
    def due(self): return self.col.sched.deck_due_tree(self.id)
    
    @property
    def cards(self): return self.col.find_cards(f"deck:{self.name}")
    
    def _repr_markdown_(self):
        n, l, r = self.due
        return f"**{self.name}**: {n} new, {l} learn, {r} review"

Wrapper class for a deck with convenient `.add()`, `.due` (new/learn/review counts), and `.cards` (list of card ids).

In [ ]:
#| export
@patch
def __getitem__(self:DeckManager, name): return Deck(self.col, name)

In [ ]:
#| export
@patch
def add(self:Collection, model=None, deck=None, tags=None, **fields):
    "Add a note with the given fields."
    model, deck = model or anki_defaults['model'], deck or anki_defaults['deck']
    note = self.new_note(self.models[model])
    for k,v in fields.items(): note[k] = v
    if tags: note.tags = tags if isinstance(tags, list) else [tags]
    res = self.add_note(note, self.decks.id_for_name(deck))
    if res.count == 0: raise ValueError("Failed to add note")
    return note

Convenient method to add a note with keyword arguments for fields. Defaults to Basic model and Default deck.

In [ ]:
note2 = col.add(Front='hola', Back='hello', tags=['spanish'])

In [ ]:
#| export
@patch
def add_deck(self:Collection, name):
    "Create a deck (use :: for nesting)."
    return self.decks.add_normal_deck_with_name(name)

Creates a new deck. Use `::` for nested decks (e.g. `"Spanish::Vocab"` creates Vocab inside Spanish).

In [ ]:
col.add_deck('Spanish::Vocab')

changes {
  deck: true
  browser_table: true
  browser_sidebar: true
  study_queues: true
  mtime: true
}
id: 1772224367444

In [ ]:
deck = col.decks['Spanish::Vocab']
deck.add(Front='adiós', Back='goodbye')
deck.due

**Vocab**: 1 new, 0 learn, 0 review

In [ ]:
col.remove_notes([note2.id])

✓ 1 change(s)

In [ ]:
col.decks.remove([col.decks.id_for_name('Spanish')])

✓ 1 change(s)

In [ ]:
col.sync()

**Sync**: NO_CHANGES

In [ ]:
#| export
@patch
def __repr__(self:ModelManager): return f"ModelManager({[m.name for m in self.all_names_and_ids()]})"

@patch
def __repr__(self:NotetypeNameId): return f"NotetypeNameId({self.id!r}, {self.name!r})"

@patch
def __repr__(self:DeckManager): return f"DeckManager({[d.name for d in self.all_names_and_ids()]})"

@patch
def __repr__(self:DeckNameId): return f"DeckNameId({self.id!r}, {self.name!r})"

@patch
def __repr__(self:DeckTreeNode):
    return f"DeckTreeNode({self.deck_id!r}, {self.name!r}, new={self.new_count}, learn={self.learn_count}, review={self.review_count})"

@patch
def __repr__(self:OpChangesWithCount): return f"OpChangesWithCount({self.count})"

@patch
def __repr__(self:Card): return f"Card({self.id}, nid={self.nid}, due={self.due}, ivl={self.ivl}, queue={self.queue})"

@patch
def __repr__(self:Deck): return f"Deck({self.name!r}, id={self.id})"

In [ ]:
col.close()

## Functional API

In [ ]:
#| export
@patch
def __enter__(self:Collection): return self

@patch
def __exit__(self:Collection, *args): self.close()

You can use `Collection` with Python's `with` statement. This ensures the collection is always properly closed, even if an error occurs during your operations. No more orphaned database locks or forgotten cleanup calls.

In [ ]:
with Collection.open() as col: note = col.add(Front='hola', Back='hello')

In [ ]:
#| export
def add_card(profile=None, model=None, deck=None, tags=None, **fields):
    "Add a card."
    profile, model, deck = profile or anki_defaults['profile'], model or anki_defaults['model'], deck or anki_defaults['deck']
    with Collection.open(profile) as col: return col.add(model=model, deck=deck, tags=tags or None, **fields)

`add_card` lets you create a new card with a single function call. Just pass your field values as keyword arguments. By default it uses the Basic note type and Default deck, but you can specify any model, deck, or tags you like.

In [ ]:
notezh = add_card(Front='你好', Back='hello')

In [ ]:
#| export
def add_fb_card(front:str, back:str, profile:str=None, model:str=None, deck:str=None, tags:str=None):
    "Add a card with a `Front` and `Back` and return the id."
    return add_card(profile=profile, model=model, deck=deck, tags=tags, Front=front, Back=back).id

In [ ]:
#| export
def find_cards(query:str, profile:str=None):
    "Find cards matching query string. Returns list of Note objects."
    profile = profile or anki_defaults['profile']
    with Collection.open(profile) as col: return [col.get_card(cid) for cid in col.find_cards(query)]

`find_cards` searches your collection and returns a list of `Card` objects. Pass any Anki search query as the first argument. Common query patterns:
- `deck:Spanish` — cards in a specific deck
- `tag:vocab` — cards with a tag
- `front:hello` — match field content
- `is:due` — cards due for review
- `added:7` — added in the last 7 days

Combine with spaces (AND) or `OR`: `deck:Spanish tag:verb` finds Spanish cards tagged "verb".

In [ ]:
cards = find_cards("deck:Default")
cards

[Card(1769220553306, nid=1769220312702, due=5, ivl=0, queue=0),
 Card(1771131052669, nid=1771131052669, due=12, ivl=0, queue=0),
 Card(1771131140690, nid=1771131140690, due=13, ivl=0, queue=0),
 Card(1771298205236, nid=1771298205236, due=14, ivl=0, queue=0),
 Card(1771994857427, nid=1771994857425, due=15, ivl=0, queue=0),
 Card(1771994862491, nid=1771994862491, due=18, ivl=0, queue=0),
 Card(1771994864613, nid=1771994864613, due=19, ivl=0, queue=0),
 Card(1771995456598, nid=1771995456598, due=22, ivl=0, queue=0),
 Card(1772223814874, nid=1772223814873, due=23, ivl=0, queue=0),
 Card(1772224020120, nid=1772224020120, due=24, ivl=0, queue=0),
 Card(1772224365000, nid=1772224365000, due=25, ivl=0, queue=0),
 Card(1772224370021, nid=1772224370021, due=28, ivl=0, queue=0),
 Card(1772224372112, nid=1772224372112, due=29, ivl=0, queue=0),
 Card(1772224372153, nid=1772224372153, due=30, ivl=0, queue=0),
 Card(1768961835197, nid=1768961835196, due=55, ivl=4, queue=2),
 Card(1769219943743, nid=1

In [ ]:
#| export
@patch
def _repr_markdown_(self:Card):
    return f"Card {self.id} (nid: {self.nid}, due: {self.due}, ivl: {self.ivl}d, queue: {self.queue})"

In [ ]:
cards[0]

Card 1769220553306 (nid: 1769220312702, due: 5, ivl: 0d, queue: 0)

In [ ]:
#| export
def find_card_ids(query:str, profile:str=None):
    "Find card ids matching query string."
    profile = profile or anki_defaults['profile']
    with Collection.open(profile) as col: return col.find_cards(query)

In [ ]:
find_card_ids("deck:Default")

[1769220553306, 1771131052669, 1771131140690, 1771298205236, 1771994857427, 1771994862491, 1771994864613, 1771995456598, 1772223814874, 1772224020120, 1772224365000, 1772224370021, 1772224372112, 1772224372153, 1768961835197, 1769219943743, 1769144854211, 1769220066429, 1769220312702]

In [ ]:
#| export
def find_notes(query:str, profile:str=None):
    "Find notes matching query string. Returns list of Note objects."
    profile = profile or anki_defaults['profile']
    with Collection.open(profile) as col: return [col.get_note(nid) for nid in col.find_notes(query)]

`find_notes` searches your collection and returns a list of `Note` objects (rather than `Card` objects). The query language is the same as `find_cards` — all the same search patterns work. The difference is that `find_notes` returns one result per note, while `find_cards` may return multiple cards if a note generates more than one card (e.g., with Cloze or Basic-and-Reversed note types).

In [ ]:
notes = find_notes("hello")
notes

[Note(1771994864613, Front='hola', Back='hello', tags=[]),
 Note(1772224372112, Front='hola', Back='hello', tags=[]),
 Note(1772224372153, Front='你好', Back='hello', tags=[])]

In [ ]:
note = notes[0]
note

**Front**: hola | **Back**: hello

In [ ]:
#| export
def find_note_ids(query:str, profile:str=None):
    "Find note ids matching query string."
    profile = profile or anki_defaults['profile']
    with Collection.open(profile) as col: return col.find_notes(query)

In [ ]:
find_note_ids("hello")

[1771994864613, 1772224372112, 1772224372153]

In [ ]:
#| export
def update_note(note:int, profile:str=None, tags:str=None, add_tags:str=None, **fields):
    "Update an existing note's fields and/or tags. Pass a Note object or note ID."
    profile = profile or anki_defaults['profile']
    with Collection.open(profile) as col:
        if not isinstance(note, Note): note = col.get_note(note)
        for k,v in fields.items(): note[k] = v
        if tags: note.tags = listify(tags)
        note.tags = list(set(listify(note.tags) + listify(add_tags)))
        col.update_note(note)
        return note

`update_note` modifies an existing note's fields and/or tags. Pass either a `Note` object or a note ID, along with any fields you want to change as keyword arguments. For tags:
- `tags=['a','b']` — replaces all tags
- `add_tags='newtag'` — adds without removing existing tags

In [ ]:
update_note(note, Back="updated answer", tags='testtag')

**Front**: hola | **Back**: updated answer | 🏷️ testtag

In [ ]:
update_note(note, add_tags='moretagz')

**Front**: hola | **Back**: updated answer | 🏷️ moretagz, testtag

In [ ]:
#| export
def update_fb_note(note_id:int, front:str='', back:str='', profile:str=None, tags:str=None, add_tags:str=None):
    "Update an existing note's front/back fields and/or tags. Pass a Note object or note ID."
    kw = {}
    if front: kw['Front'] = front
    if back: kw['Back'] = back
    return update_note(note_id, profile=profile, tags=tags, add_tags=add_tags, **kw)

In [ ]:
update_fb_note(note, front='I am new')

**Front**: I am new | **Back**: updated answer | 🏷️ moretagz, testtag

In [ ]:
#| export
def get_note(note_id:int, profile:str=None):
    "Retrieve a note by ID."
    profile = profile or anki_defaults['profile']
    with Collection.open(profile) as col: return col.get_note(note_id)

In [ ]:
get_note(note.id)

**Front**: I am new | **Back**: updated answer | 🏷️ moretagz, testtag

In [ ]:
#| export
def del_card(notes:int, profile:str=None):
    "Delete card(s) by Note(s) or note id(s)."
    profile = profile or anki_defaults['profile']
    nids = [n if isinstance(n, int) else n.id for n in listify(notes)]
    with Collection.open(profile) as col: return col.remove_notes(nids)

In [ ]:
del_card([notezh, note])

✓ 2 change(s)

In [ ]:
#| export
def sync(profile:str=None, user:str=None, passw:str=None, media:bool=True):
    "Sync collection, handling open/close automatically."
    profile = profile or anki_defaults['profile']
    with Collection.open(profile) as col: return col.sync(user=user, passw=passw, media=media)

`sync` handles the entire sync lifecycle for you — opening the collection, authenticating with AnkiWeb, syncing, and closing up afterwards. The first time you sync, pass your AnkiWeb credentials; they'll be saved for future use.

In [ ]:
o = sync(user=user, passw=passw) # First time
# sync()  # after that

In [ ]:
#| export
def anki_tools(): print('&`[add_fb_card, find_notes, find_note_ids, find_cards, find_card_ids, get_note, del_card, update_fb_note, sync]`')

Adding cloze cards

In [ ]:
#| export
def add_cloze_card(text, deck='Default', tags=None, back_extra=''):
    """Add a cloze deletion card."""
    note = add_card(model='Cloze', deck=deck, tags=tags, **{'Text': text, 'Back Extra': back_extra})
    return note.id

In [ ]:
test_cloze = add_cloze_card("Minus times {{c1::minus}} is always {{c2::plus}}", back_extra="Proof. Let –𝑎 and –𝑏 represent any two negative numbers. Then (−𝑎)(−𝑏)=(−1)(𝑎)(−1)(𝑏)=𝑎𝑏(−1)(−1)=𝑎𝑏.")


In [ ]:
get_note(test_cloze)

**Text**: Minus times {{c1::minus}} is always {{c2::plus}} | **Back Extra**: Proof. Let –𝑎 and –𝑏 represent any two negative numbers. Then (−𝑎)(−𝑏)=(−1)(𝑎)(−1)(𝑏)=𝑎𝑏(−1)(−1)=𝑎𝑏.

In [ ]:
update_note(test_cloze, add_tags='precalculus')

**Text**: Minus times {{c1::minus}} is always {{c2::plus}} | **Back Extra**: Proof. Let –𝑎 and –𝑏 represent any two negative numbers. Then (−𝑎)(−𝑏)=(−1)(𝑎)(−1)(𝑏)=𝑎𝑏(−1)(−1)=𝑎𝑏. | 🏷️ precalculus

In [ ]:
get_note(test_cloze)

**Text**: Minus times {{c1::minus}} is always {{c2::plus}} | **Back Extra**: Proof. Let –𝑎 and –𝑏 represent any two negative numbers. Then (−𝑎)(−𝑏)=(−1)(𝑎)(−1)(𝑏)=𝑎𝑏(−1)(−1)=𝑎𝑏. | 🏷️ precalculus

In [ ]:
del_card(test_cloze)

✓ 2 change(s)